# Decorators

The most common way to apply metaprogramming in Python.
You can add a piece of code that handles a function call.

By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

In Python, functions are first-class objects. This means that functions can be passed around and used as arguments.

It’s possible to define functions inside other functions. Such functions are called inner functions. Here’s an example of a function with two inner functions:

In [20]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()
# Let's call it
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Note that the order in which the inner functions are defined does not matter. Like with any other functions, the printing only happens when the inner functions are executed.

Furthermore, the inner functions are not defined until the parent function is called. They are locally scoped to parent(): they only exist inside the parent() function as local variables. Try calling first_child(). You should get an error.

Python also allows you to use functions as return values. The following example returns one of the inner functions from the outer parent() function:

In [ ]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

print(parent(1)) # We are trying to print a function
# Let's assign the functions to variables
first = parent(1)
second = parent(2)
# We can call them as normal functions
print(first()) 
print(second())

## Simple Decorators
Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

# Let's call it
say_whee()

The name say_whee now points to the wrapper() inner function. Remember that you return wrapper as a function when you call my_decorator(say_whee).

Because wrapper() is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [ ]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

say_whee()

Python allows you to use decorators in a simpler way with the @ symbol, sometimes called the “pie” syntax. 

The following example does the exact same thing as the first decorator example:

In [ ]:
@my_decorator
def say_whee():
    print("Whee!")

say_whee()

## Decorating functions with arguments

When you have a function with parameters, can you still decorate it? Let's see:

In [ ]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

greet("World")

The solution is to use *args and **kwargs in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. 

Let's rewrite it:

In [ ]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print('Whee!')

@do_twice
def greet(name):
    print(f"Hello {name}")

greet("World")
say_whee()

## Returning values from decorated functions

If we want to return a value, as the decorator wraps the function, you need to return it from it. 


In [ ]:
# This won't work after previous code snippet
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

return_greeting("Socialpoint")

In [ ]:
# This should work

def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

# Now Run again the previous code ^^

## Some real world examples
Now we will try to see some real examples of decorators and more advanced behaviours. 
Let's see them first and then play with diferent calls.

In [15]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func) # <- See the reference for more info about this line
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"TIMER - Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"DEBUG - Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"DEBUG - {func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

PLUGINS = dict()

# We do not need to call the function in the decorator
def register(func):
    """ Register a function as a plug-in """
    PLUGINS[func.__name__] = func
    return func

# A decorator can accept parameters
def repeat(num_times):
    """ Repeat multiple times a function """
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

    

In [2]:
# We can nest multiple decorators at once, order is important
@timer
@debug
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

import math

@debug
def approximate_e(terms=18):
    """Calculate math constant e by aproximation"""
    return sum(1 / math.factorial(n) for n in range(terms))

waste_some_time(200)

print(make_greeting("Jordi"))
print(make_greeting("Xavi",73))

print(approximate_e(4))

# We also can apply a decorator to a standard library function
# math.factorial = debug(math.factorial)

# print(approximate_e(10))


Calling waste_some_time(200)
'waste_some_time' returned None
Finished 'waste_some_time' in 0.4214 secs
Calling make_greeting('Jordi')
'make_greeting' returned 'Howdy Jordi!'
Howdy Jordi!
Calling make_greeting('Xavi', 73)
'make_greeting' returned 'Whoa Xavi! 73 already, you are growing up!'
Whoa Xavi! 73 already, you are growing up!
Calling approximate_e(4)
'approximate_e' returned 2.6666666666666665
2.6666666666666665


In [19]:
@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

make_greeting = register(make_greeting)



import random

#@repeat(num_times=3)
def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

print(randomly_greet("Xavi"))
# print(randomly_greet("Xavi"))
# print(randomly_greet("Xavi"))

Using 'say_hello'
Using 'be_awesome'
Using 'make_greeting'
Calling make_greeting('Xavi')
'make_greeting' returned 'Howdy Xavi!'
Howdy Xavi!
Using 'make_greeting'
Calling make_greeting('Xavi')
'make_greeting' returned 'Howdy Xavi!'
Using 'make_greeting'
Calling make_greeting('Xavi')
'make_greeting' returned 'Howdy Xavi!'
Using 'make_greeting'
Calling make_greeting('Xavi')
'make_greeting' returned 'Howdy Xavi!'
Howdy Xavi!
Using 'make_greeting'
Calling make_greeting('Xavi')
'make_greeting' returned 'Howdy Xavi!'
Using 'say_hello'
Using 'say_hello'
Hello Xavi


## Some real usages for decorators

As we have seen decorators give a way to manipulate how functions and classes work (yes, a class can also be decorated). Some typical usage for that is:
- Registering functions: As we have seen, we can attach functions to a Web Framework
  - Typically we can attach our own functions to API REST calls
- Singleton classes: A class that can only be instantiated once
- Cache return values: We can save a function result to avoid its calculation over and over.
- Validating inputs: Working with JSON Schema or similar, we can validate that an input json is valid outside our function
